## Deliverable 2. Create a Customer Travel Destinations Map.

In [62]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

import sys
import os




In [63]:
# Import API key
#import path
sys.path.append('../World_Weather_Analysis/')

from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)


In [64]:
# 1. Import the WeatherPy_database.csv file. 
#city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Haines Junction,CA,60.7522,-137.5108,46.35,71,96,2.30,overcast clouds
1,1,Dzhebariki-Khaya,RU,62.2167,135.8000,37.72,97,100,3.96,overcast clouds
2,2,Tuatapere,NZ,-46.1333,167.6833,43.48,80,85,5.41,overcast clouds
3,3,Oranjemund,NaN,-28.5500,16.4333,57.85,84,8,6.31,clear sky
4,4,Nikolskoye,RU,59.7035,30.7861,53.49,91,62,8.68,broken clouds


In [65]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [66]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
11,11,Tlacotalpan,MX,18.6167,-95.6667,76.48,93,100,8.79,overcast clouds
12,12,Kapaa,US,22.0752,-159.3190,83.10,72,20,8.05,few clouds
16,16,Akyab,MM,20.1500,92.9000,78.51,86,40,2.06,scattered clouds
24,24,Cabras,IT,39.9300,8.5314,77.95,82,100,6.51,overcast clouds
29,29,Kavieng,PG,-2.5744,150.7967,81.10,76,100,14.03,light rain
31,31,Sabang,ID,5.8933,95.3214,81.09,77,100,9.28,overcast clouds
33,33,Micheweni,TZ,-4.9667,39.8333,75.09,83,1,11.61,clear sky
34,34,Avera,US,33.1940,-82.5271,83.55,33,0,8.10,clear sky
37,37,Alofi,NU,-19.0595,-169.9187,75.09,94,82,5.75,broken clouds
40,40,Pilar,BR,-9.5972,-35.9567,75.13,81,45,6.08,scattered clouds


In [67]:
# 4a. Determine if there are any empty rows.
#preferred_cities_df.count()
preferred_cities_df.isnull().sum()


City_ID                0
City                   0
Country                1
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [68]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = preferred_cities_df.dropna()
clean_df = preferred_cities_df

In [69]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
11,Tlacotalpan,MX,76.48,overcast clouds,18.6167,-95.6667,
12,Kapaa,US,83.10,few clouds,22.0752,-159.3190,
16,Akyab,MM,78.51,scattered clouds,20.1500,92.9000,
24,Cabras,IT,77.95,overcast clouds,39.9300,8.5314,
29,Kavieng,PG,81.10,light rain,-2.5744,150.7967,
31,Sabang,ID,81.09,overcast clouds,5.8933,95.3214,
33,Micheweni,TZ,75.09,clear sky,-4.9667,39.8333,
34,Avera,US,83.55,clear sky,33.1940,-82.5271,
37,Alofi,NU,75.09,broken clouds,-19.0595,-169.9187,
40,Pilar,BR,75.13,scattered clouds,-9.5972,-35.9567,


In [70]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [72]:
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
11,Tlacotalpan,MX,76.48,overcast clouds,18.6167,-95.6667,Hotel Casa Blanca
12,Kapaa,US,83.10,few clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
16,Akyab,MM,78.51,scattered clouds,20.1500,92.9000,ဟိုတယ် မန်မိုရီ
24,Cabras,IT,77.95,overcast clouds,39.9300,8.5314,Hotel Villa Canu
29,Kavieng,PG,81.10,light rain,-2.5744,150.7967,Nusa Island Retreat
...,...,...,...,...,...,...,...
638,Nisia Floresta,BR,81.41,scattered clouds,-6.0911,-35.2086,Rancho Crepusculo
641,Red Wing,US,85.78,clear sky,44.5625,-92.5338,"St. James Hotel, 1875 - A Historic Hotel of Am..."
642,Acapulco,MX,85.82,broken clouds,16.8634,-99.8901,HS Hotsson Smart Acapulco
653,Meadow Lake,US,82.40,clear sky,34.8014,-106.5436,TumbleWeed RV Park


In [73]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df=hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
11,Tlacotalpan,MX,76.48,overcast clouds,18.6167,-95.6667,Hotel Casa Blanca
12,Kapaa,US,83.10,few clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
16,Akyab,MM,78.51,scattered clouds,20.1500,92.9000,ဟိုတယ် မန်မိုရီ
24,Cabras,IT,77.95,overcast clouds,39.9300,8.5314,Hotel Villa Canu
29,Kavieng,PG,81.10,light rain,-2.5744,150.7967,Nusa Island Retreat
...,...,...,...,...,...,...,...
638,Nisia Floresta,BR,81.41,scattered clouds,-6.0911,-35.2086,Rancho Crepusculo
641,Red Wing,US,85.78,clear sky,44.5625,-92.5338,"St. James Hotel, 1875 - A Historic Hotel of Am..."
642,Acapulco,MX,85.82,broken clouds,16.8634,-99.8901,HS Hotsson Smart Acapulco
653,Meadow Lake,US,82.40,clear sky,34.8014,-106.5436,TumbleWeed RV Park


In [74]:
# 8a. Create the output File (CSV)
output_data_file="WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [75]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [76]:
# 11a. Add a marker layer for each city to the map. 

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))